In [0]:
%pip install pandas

In [0]:
import pandas as pd
import numpy as np
import numpy as np
import sqlite3

# Read the CSV file using pandas
file_path = "./history.csv"
df = pd.read_csv(file_path)
df.info()

In [0]:
df.head().transpose()

In [0]:
df.head()

- Coluna 	TotalCharges possui 278 linhas vazias (NaN), provavelmente novos clientes que tem tempo 0
- Coluna Tenure está como object, precisa ransformar para float64, ela possui espaços em branco impedindo a leitura altomática
- Conferir quais coluna possuem "No internet service", matematicamente isso é a mesma coisa que um "No", conferir quais colunas tem esse valor e substituir. Ex de coluna: OnlineSecurity, TechSupport,
- Verificação do desbalanceamento do churn, baseline que precisamos barter

In [0]:
#Coluna TotalCharges
# Mostra apenas as linhas onde TotalCharges está vazio
filtro_vazios = df[df['TotalCharges'].isna()]
#print(filtro_vazios[['tenure', 'MonthlyCharges', 'TotalCharges']].head())

#TotalCharges = tenure x MonthlyCharges
#Converter tenure para numérico
df['tenure'] = pd.to_numeric(df['tenure'], errors='coerce')

#Preencher os NaNs de TotalCharges usando a multiplicação
df['TotalCharges'] = df['TotalCharges'].fillna(df['tenure'] * df['MonthlyCharges'])

#Verificação final
print(f"TotalCharges vazios restantes: {df['TotalCharges'].isna().sum()}")
print(df.loc[79, ['tenure', 'MonthlyCharges', 'TotalCharges']])
#df.info()

#Remover a última linha com dados faltantes
df.dropna(subset=['TotalCharges'], inplace=True)

In [0]:
#Substituindo No Internet por No 
colunas = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']
for col in colunas:
    df[col] = df[col].replace({'No internet service': 'No', 'No phone service': 'No'})
df.head()

In [0]:
#Churn
#Yes = 1 e No = 0
df['Churn'] = df['Churn'].astype(str).str.strip()
df['Churn'] = np.where(df['Churn'].str.lower() == 'yes', 1, 0)
print(df['Churn'].value_counts())


In [0]:
print(df['Churn'].value_counts(normalize=True) * 100)

Não sairam 73.317948%
Sairam 26.682052%
Meta mínima de baseline é 73%

In [0]:
#salvando sql e csv

df.to_csv('dados_churn_limpos.csv', index=False)
print("Arquivo CSV salvo com sucesso!")

#sql
conexao = sqlite3.connect('telecom_churn.db')
df.to_sql('clientes', conexao, if_exists='replace', index=False)
conexao.close()
print("Banco de dados (.db) salvo com sucesso!")
